# Visualizing what convolutional networks learn


![](./figs/alexnet_filters.png) 


Resources


- Zeiler and Fergus paper 

  https://cs.nyu.edu/~fergus/papers/zeilerECCV2014.pdf

- The building blocks of interpretability

  https://distill.pub/2018/building-blocks/

- A nice video from Yosinski et al.

  https://www.youtube.com/watch?v=AgkfIQ4IGaM&t=132s

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary

import numpy as np
from matplotlib import pyplot as plt

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
    def extract(self,x):
        '''
        Extract representations from input, first hidden layer and max pooling
        '''
        h0 = x
        h1 = F.relu(self.conv1(x))
        h2 = F.max_pool2d(h1, 2, 2)
        return [h0,h1,h2]      

In [ ]:
# Training settings
input_size=(1,28,28,)
batch_size=64
test_batch_size=1000
epochs=1
lr=0.01
momentum=0.0   
seed=1
log_interval=100

In [ ]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [ ]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [ ]:
summary(model,input_size)

# Guided exercise

We will now visualize the learned features in a convolutional network. 

We will

- load the trained model
- look once again at the model architecture 
  and focus on the first conv layer 
  
  Question: how many filters/feature maps do we have? 
  
- extract these filters

Making sense of artificial units specialization in deeper layers typically
require optimization.

We will not cover it but you will find a nice example in Keras https://keras.io/examples/conv_filter_visualization/
where, through an optimization process in input space you can
visualize what neurons in deep layers (beyond the first convolutional layer)
*prefer*.

A picture here: these are the images preferred by some units

![](./figs/vgg_block5_conv1.png) 



In [ ]:
model.load_state_dict(torch.load('mnist_cnn.pt'))
model.eval()

In [ ]:
for p in model.parameters():
    print(p.shape)

In [ ]:
inputs,labels = next(iter(test_loader))
output = model(inputs)
pred = output.argmax(dim=1, keepdim=True) 
correct = pred.eq(labels.view_as(pred)).sum().item()
print('acc = {}'.format(correct/inputs.shape[0]) )

In [ ]:
fig=plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(np.squeeze(inputs[i]),cmap='bone')
    plt.title(str(pred[i].item()))
    plt.xticks([])
    plt.yticks([])

In [ ]:
lay1_filters = list(model.parameters())[0].detach().numpy()

In [ ]:
lay1_filters.shape

In [ ]:
fig=plt.figure(figsize=(10,10))
for i in range(20):
    plt.subplot(4,5,i+1)
    plt.imshow(np.squeeze(lay1_filters[i]),cmap='bone')
    plt.title(i)
    plt.xticks([])
    plt.yticks([])

We now extract representations because we want to see how the network transformed one image at the first hidden layer.

http://setosa.io/ev/image-kernels/

In [ ]:
[inp, conv1, maxpool1] = model.extract(inputs)

In [ ]:
conv1.shape

In [ ]:
conv1[0].shape

In [ ]:
x = conv1[0].detach().numpy()

In [ ]:
fig=plt.figure(figsize=(10,10))
for i in range(20):
    plt.subplot(4,5,i+1)
    plt.imshow(x[i],cmap='bone')
    plt.title(i)
    plt.xticks([])
    plt.yticks([])

Visualization of filters in a *real* network: AlexNet

Maybe you will have to run

    $ conda install ipywidgets
    
in your environemnt, if you experience an error in the next cell

In [ ]:
from torchvision.models import alexnet
print('(The first time you download the pretrained model it will take a while...)')
alex = alexnet(pretrained=True)

In [ ]:
for p in alex.parameters():
    print(p.shape)

In [ ]:
lay1_filters_alex = list(alex.parameters())[0].detach().numpy()
print(lay1_filters_alex.shape)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
shape = np.squeeze(lay1_filters_alex[0]).shape
new_shape = np.array(shape)
new_shape = new_shape[[1,2,0]]
new_shape

In [ ]:
np.min(lay1_filters_alex[i].transpose(2,1,0))

In [ ]:
def normalize(x):
    x -= np.min(x)
    x /= np.max(x)
    return x

In [ ]:
fig=plt.figure(figsize=(20,20))
for i in range(64):
    plt.subplot(8,8,i+1)
    plt.imshow(normalize(lay1_filters_alex[i].transpose(2,1,0)))
    plt.title(i)
    plt.xticks([])
    plt.yticks([])